Information need to retrieve:
1. Author’s association with repository. (author_association)

In [1]:
import numpy as np
import pandas as pd
import re
import requests
import json
from datetime import datetime

In [ ]:
df_quote = pd.read_csv('ToulminArgumentationModel.csv')

In [3]:
# remove all time_author
display(df_quote.shape)

df_quote = df_quote.loc[((df_quote['Codes'] != 'time_author') & 
                         (df_quote['Codes'] != 'user_study_cutoff') & 
                         (df_quote['Codes'] != 'here'))]

display(df_quote.shape)

(6330, 5)

(5142, 5)

In [4]:
df_time_author = pd.read_excel('TimeAuthor.xlsx', sheet_name='Sheet1', usecols =['Text Content', 'Document', 'Codes', 'Start position', 'End position'])

In [1]:
# get author_association
access_token = 'xxx' # removed access token for anonymity

def GetAuthorAssociation(row):
    doc_name = row['Document']
    timestamp = row['time_cur']
    author = row['author_cur']
    print(doc_name + ' ' + timestamp + ' ' + author)
    issue_num = re.findall('\d+',doc_name)
    request_str = "https://api.github.com/repos/microsoft/vscode/issues/" + issue_num[0] + "/comments?since=" + timestamp + "&access_token=" + access_token
#     print(request_str)
    r = requests.get(request_str)
    author_association = 'NOT FOUND'
    try:
        commentList = json.loads(r.text or r.content)
        for comment in commentList:
    #         print(comment['user']['login'])
#             print(author)
    #         print(comment['created_at'])
    #         print(timestamp)
            if (comment['user']['login'] == author) & (comment['created_at'] == timestamp):
                author_association = comment['author_association']
                break
    except:
        print('EXCEED ACCESS LIMIT')
        author_association ='EXCEED ACCESS LIMIT'
#     print(author_association)
    return author_association

In [6]:
# get time and author
def GetTimeAuthor(row):
    doc_name = row['Document']
    start_pos = row['Start position']
    end_pos = row['End position']
    df_temp_doc = df_time_author.loc[(df_time_author['Document'] == doc_name)]
    # current time and author
    df_temp_previous = df_temp_doc.loc[df_temp_doc['End position'] < start_pos]
    df_temp_previous = df_temp_previous.sort_values('Start position', ascending=False).head(n=2)
    if len(df_temp_previous) > 0:
        df_temp_cur = df_temp_previous.iloc[0]
        temp_time_author_cur = df_temp_cur['Text Content'].split()
        temp_time_cur =  temp_time_author_cur[0]
        temp_author_cur = temp_time_author_cur[1]
    else:
        temp_time_cur = ''
        temp_author_cur = ''
    if len(df_temp_previous) > 1:
        df_temp_pre = df_temp_previous.iloc[1]
        temp_time_author_pre = df_temp_pre['Text Content'].split()
        temp_time_pre =  temp_time_author_pre[0]
        temp_author_pre = temp_time_author_pre[1]
    else:
        temp_time_pre = ''
        temp_author_pre = ''
    # get next time and author
    df_temp_next = df_temp_doc.loc[df_temp_doc['Start position'] > end_pos]
#     display(df_temp_next)
    if len(df_temp_next) > 0:
        df_temp_next = df_temp_next.loc[df_temp_next['Start position'].idxmin()]
        temp_time_author_next = df_temp_next['Text Content'].split()
        temp_time_next = temp_time_author_next[0]
        temp_author_next = temp_time_author_next[1]
    else:
        temp_time_next = ''
        temp_author_next = ''
    return temp_time_pre, temp_author_pre, temp_time_cur, temp_author_cur, temp_time_next, temp_author_next

In [7]:
df_quote[['time_pre', 'author_pre', 'time_cur', 'author_cur', 'time_next', 'author_next']] = df_quote.apply(GetTimeAuthor, axis=1, result_type='expand')

In [8]:
df_quote['author_association'] = df_quote.apply(GetAuthorAssociation, axis=1)
# this will run without error after given valid access_token

issue9388_allCommentsText.doc 2016-07-16T14:48:59Z ivalexa
issue9388_allCommentsText.doc 2016-07-16T14:48:59Z ivalexa
issue9388_allCommentsText.doc 2016-07-16T16:20:03Z brunnopleffken
issue9388_allCommentsText.doc 2016-07-16T16:44:34Z ivalexa
issue9388_allCommentsText.doc 2016-07-16T22:02:25Z bgse
issue9388_allCommentsText.doc 2016-07-16T22:02:25Z bgse
issue9388_allCommentsText.doc 2016-07-21T04:55:50Z payonel
issue9388_allCommentsText.doc 2016-07-21T04:55:50Z payonel
issue9388_allCommentsText.doc 2016-07-21T04:55:50Z payonel
issue9388_allCommentsText.doc 2016-07-21T04:55:50Z payonel
issue9388_allCommentsText.doc 2016-07-21T04:55:50Z payonel
issue9388_allCommentsText.doc 2016-08-23T04:17:22Z iamsunny
issue9388_allCommentsText.doc 2016-08-23T04:17:22Z iamsunny
issue9388_allCommentsText.doc 2016-08-23T04:17:22Z iamsunny
issue9388_allCommentsText.doc 2016-10-31T04:31:01Z azizsaboor
issue9388_allCommentsText.doc 2016-10-31T04:31:01Z azizsaboor
issue9388_allCommentsText.doc 2016-10-31T04:31

issue9388_allCommentsText.doc 2018-01-15T13:11:55Z KangZiRu
issue9388_allCommentsText.doc 2018-01-15T13:11:55Z KangZiRu
issue9388_allCommentsText.doc 2018-01-22T04:54:41Z ponylin1985
issue9388_allCommentsText.doc 2018-01-22T04:54:41Z ponylin1985
issue9388_allCommentsText.doc 2018-01-22T04:54:41Z ponylin1985
issue9388_allCommentsText.doc 2018-01-22T04:54:41Z ponylin1985
issue9388_allCommentsText.doc 2018-01-23T08:39:44Z fractalf
issue9388_allCommentsText.doc 2018-01-23T08:39:44Z fractalf
issue9388_allCommentsText.doc 2018-01-23T08:55:03Z finalclass
issue9388_allCommentsText.doc 2018-01-23T08:55:03Z finalclass
issue9388_allCommentsText.doc 2018-01-23T08:55:03Z finalclass
issue9388_allCommentsText.doc 2018-01-23T08:55:03Z finalclass
issue9388_allCommentsText.doc 2018-01-23T08:55:03Z finalclass
issue9388_allCommentsText.doc 2018-01-23T08:55:03Z finalclass
issue9388_allCommentsText.doc 2018-01-23T08:55:03Z finalclass
issue9388_allCommentsText.doc 2018-01-23T08:55:03Z finalclass
issue9388_al

issue9388_allCommentsText.doc 2017-01-06T06:39:49Z bpasero
issue9388_allCommentsText.doc 2017-01-06T06:39:49Z bpasero
issue9388_allCommentsText.doc 2017-04-25T21:10:01Z aight8
issue9388_allCommentsText.doc 2017-10-11T08:23:03Z 2289034325
issue9388_allCommentsText.doc 2017-11-18T07:29:16Z giangcoi48k
issue9388_allCommentsText.doc 2016-07-16T14:48:59Z ivalexa
issue9388_allCommentsText.doc 2016-07-16T14:48:59Z ivalexa
issue9388_allCommentsText.doc 2016-07-21T04:55:50Z payonel
issue9388_allCommentsText.doc 2017-08-22T15:22:46Z chriscarpenter12
issue9388_allCommentsText.doc 2017-08-22T15:22:46Z chriscarpenter12
issue9388_allCommentsText.doc 2017-10-23T14:16:36Z Montago
issue9388_allCommentsText.doc 2018-01-23T08:39:44Z fractalf
issue9388_allCommentsText.doc 2018-01-23T08:55:03Z finalclass
issue9388_allCommentsText.doc 2018-01-23T08:55:03Z finalclass
issue9388_allCommentsText.doc 2018-01-29T11:15:39Z montapro
issue9388_allCommentsText.doc 2018-01-31T11:39:01Z tvvignesh
issue9388_allCommentsT

issue4865_allCommentsText.doc 2017-01-23T15:54:58Z jfcartier
issue4865_allCommentsText.doc 2017-01-23T15:54:58Z jfcartier
issue4865_allCommentsText.doc 2017-01-23T15:54:58Z jfcartier
issue4865_allCommentsText.doc 2017-01-23T15:54:58Z jfcartier
issue4865_allCommentsText.doc 2017-01-23T15:54:58Z jfcartier
issue4865_allCommentsText.doc 2017-02-06T09:57:03Z idears-org
issue4865_allCommentsText.doc 2017-02-11T13:33:51Z Phrohdoh
issue4865_allCommentsText.doc 2017-02-11T13:33:51Z Phrohdoh
issue4865_allCommentsText.doc 2017-02-11T13:33:51Z Phrohdoh
issue4865_allCommentsText.doc 2017-02-17T09:45:35Z heliopassarelli
issue4865_allCommentsText.doc 2017-02-17T16:01:59Z sapiraz
issue4865_allCommentsText.doc 2017-02-17T20:39:52Z gamedevsam
issue4865_allCommentsText.doc 2017-02-17T20:39:52Z gamedevsam
issue4865_allCommentsText.doc 2017-02-17T20:39:52Z gamedevsam
issue4865_allCommentsText.doc 2017-02-17T20:39:52Z gamedevsam
issue4865_allCommentsText.doc 2017-02-17T20:39:52Z gamedevsam
issue4865_allComm

issue4865_allCommentsText.doc 2017-01-22T16:35:44Z druellan
issue4865_allCommentsText.doc 2017-01-23T15:22:15Z sapiraz
issue4865_allCommentsText.doc 2017-01-23T15:22:15Z sapiraz
issue4865_allCommentsText.doc 2017-01-23T15:35:25Z martin-wiseweb
issue4865_allCommentsText.doc 2017-01-23T15:35:25Z martin-wiseweb
issue4865_allCommentsText.doc 2017-01-23T15:39:54Z lankymart
issue4865_allCommentsText.doc 2017-01-23T15:39:54Z lankymart
issue4865_allCommentsText.doc 2017-02-27T12:40:06Z FANMixco
issue4865_allCommentsText.doc 2017-02-27T12:40:06Z FANMixco
issue4865_allCommentsText.doc 2017-02-27T14:53:39Z druellan
issue4865_allCommentsText.doc 2017-02-27T14:53:39Z druellan
issue4865_allCommentsText.doc 2017-02-27T14:53:39Z druellan
issue4865_allCommentsText.doc 2017-02-27T15:04:25Z pohmelie
issue4865_allCommentsText.doc 2017-02-27T15:04:25Z pohmelie
issue4865_allCommentsText.doc 2017-02-28T06:32:09Z mika76
issue4865_allCommentsText.doc 2017-02-28T06:32:09Z mika76
issue4865_allCommentsText.doc 20

...

issue224_allCommentsText.doc 2016-04-28T11:56:33Z hsyn
issue224_allCommentsText.doc 2016-04-28T11:56:33Z hsyn
issue224_allCommentsText.doc 2016-04-28T12:42:31Z TurkeyMan
issue224_allCommentsText.doc 2016-04-28T12:42:31Z TurkeyMan
issue224_allCommentsText.doc 2016-04-28T13:44:37Z anyong
issue224_allCommentsText.doc 2016-04-28T13:44:37Z anyong
issue224_allCommentsText.doc 2016-04-28T15:34:37Z alexgorbatchev
issue224_allCommentsText.doc 2016-04-28T15:34:37Z alexgorbatchev
issue224_allCommentsText.doc 2016-04-28T15:34:37Z alexgorbatchev
issue224_allCommentsText.doc 2016-04-28T15:34:37Z alexgorbatchev
issue224_allCommentsText.doc 2016-04-28T15:34:37Z alexgorbatchev
issue224_allCommentsText.doc 2016-04-28T15:43:24Z kfranqueiro
issue224_allCommentsText.doc 2016-04-28T15:43:24Z kfranqueiro
issue224_allCommentsText.doc 2016-04-28T15:43:24Z kfranqueiro
issue224_allCommentsText.doc 2016-04-28T15:46:03Z anyong
issue224_allCommentsText.doc 2016-04-28T15:46:03Z anyong
issue224_allCommentsText.doc 20

issue224_allCommentsText.doc 2016-05-10T17:09:50Z AshleyGrant
issue224_allCommentsText.doc 2016-05-10T05:25:36Z be5invis
issue224_allCommentsText.doc 2016-05-10T05:25:36Z be5invis
issue224_allCommentsText.doc 2016-05-10T05:25:36Z be5invis
issue224_allCommentsText.doc 2016-05-10T05:30:07Z anyong
issue224_allCommentsText.doc 2016-05-10T05:30:07Z anyong
issue224_allCommentsText.doc 2016-05-10T17:14:38Z orospakr
issue224_allCommentsText.doc 2016-05-10T17:14:38Z orospakr
issue224_allCommentsText.doc 2016-05-10T17:14:38Z orospakr
issue224_allCommentsText.doc 2016-05-10T17:14:38Z orospakr
issue224_allCommentsText.doc 2016-05-10T17:14:38Z orospakr
issue224_allCommentsText.doc 2016-05-10T17:14:38Z orospakr
issue224_allCommentsText.doc 2016-05-10T17:14:38Z orospakr
issue224_allCommentsText.doc 2016-05-10T17:14:38Z orospakr
issue224_allCommentsText.doc 2016-05-10T17:14:38Z orospakr
issue224_allCommentsText.doc 2016-05-10T17:14:38Z orospakr
issue224_allCommentsText.doc 2016-05-10T17:14:38Z orospak

issue224_allCommentsText.doc 2016-06-07T16:20:10Z andradei
issue224_allCommentsText.doc 2016-06-07T16:20:10Z andradei
issue224_allCommentsText.doc 2016-06-07T16:20:10Z andradei
issue224_allCommentsText.doc 2016-06-07T16:20:10Z andradei
issue224_allCommentsText.doc 2016-06-07T16:20:10Z andradei
issue224_allCommentsText.doc 2016-06-07T16:20:10Z andradei
issue224_allCommentsText.doc 2016-06-07T16:20:10Z andradei
issue224_allCommentsText.doc 2016-06-07T19:06:34Z JoshClose
issue224_allCommentsText.doc 2016-06-07T19:06:34Z JoshClose
issue224_allCommentsText.doc 2016-06-07T19:06:34Z JoshClose
issue224_allCommentsText.doc 2016-06-07T20:00:29Z chrisdias
issue224_allCommentsText.doc 2016-06-07T20:00:29Z chrisdias
issue224_allCommentsText.doc 2016-06-07T20:00:29Z chrisdias
issue224_allCommentsText.doc 2016-06-07T20:02:22Z chrisdias
issue224_allCommentsText.doc 2016-06-07T20:02:22Z chrisdias
issue224_allCommentsText.doc 2016-06-07T23:35:09Z chrisdias
issue224_allCommentsText.doc 2016-06-07T23:35:0

issue224_allCommentsText.doc 2016-05-09T23:18:19Z OlsonDev
issue224_allCommentsText.doc 2016-05-10T03:38:23Z awkr
issue224_allCommentsText.doc 2016-05-10T13:09:01Z mattwilson1024
issue224_allCommentsText.doc 2016-05-10T13:09:01Z mattwilson1024
issue224_allCommentsText.doc 2016-05-10T19:37:20Z muellerkyle
issue224_allCommentsText.doc 2016-05-10T19:37:20Z muellerkyle
issue224_allCommentsText.doc 2016-05-10T19:37:20Z muellerkyle
issue224_allCommentsText.doc 2016-05-12T05:44:07Z Tyriar
issue224_allCommentsText.doc 2016-05-16T08:11:01Z Namek
issue224_allCommentsText.doc 2016-03-02T17:41:32Z waderyan
issue224_allCommentsText.doc 2016-03-12T13:18:47Z TurkeyMan
issue224_allCommentsText.doc 2016-04-05T14:09:59Z indiejames
issue224_allCommentsText.doc 2016-04-08T16:41:09Z Tyriar
issue224_allCommentsText.doc 2016-05-09T21:43:57Z MrTravisB
issue224_allCommentsText.doc 2016-05-10T11:06:23Z be5invis
issue224_allCommentsText.doc 2016-05-10T11:06:23Z be5invis
issue224_allCommentsText.doc 2016-03-21T22

EXCEED ACCESS LIMIT
issue224_allCommentsText.doc 2015-12-21T13:42:54Z inakianduaga
EXCEED ACCESS LIMIT
issue224_allCommentsText.doc 2015-12-23T00:11:48Z TurkeyMan
EXCEED ACCESS LIMIT
issue224_allCommentsText.doc 2015-12-23T00:11:48Z TurkeyMan
EXCEED ACCESS LIMIT
issue224_allCommentsText.doc 2015-12-23T00:11:48Z TurkeyMan
EXCEED ACCESS LIMIT
issue224_allCommentsText.doc 2015-12-23T00:11:48Z TurkeyMan
EXCEED ACCESS LIMIT
issue224_allCommentsText.doc 2015-12-23T00:45:49Z Xety
EXCEED ACCESS LIMIT
issue224_allCommentsText.doc 2015-12-23T16:00:59Z kfranqueiro
EXCEED ACCESS LIMIT
issue224_allCommentsText.doc 2015-12-24T04:22:58Z TurkeyMan
EXCEED ACCESS LIMIT
issue224_allCommentsText.doc 2016-01-03T09:42:12Z Mikewl
EXCEED ACCESS LIMIT
issue224_allCommentsText.doc 2015-12-23T08:41:40Z bpasero
EXCEED ACCESS LIMIT
issue224_allCommentsText.doc 2015-12-24T04:22:58Z TurkeyMan
EXCEED ACCESS LIMIT
issue224_allCommentsText.doc 2015-12-21T12:58:28Z snehilmodani
EXCEED ACCESS LIMIT
issue224_allCommentsTe

In [9]:
df_quote.to_csv('AuthorAssociation_data.csv', index=False)